# A script to obtain the T5 solution

Action:
$$
S_{11} = \int d^{11}x \sqrt{-g} \left[R - \frac{1}{2}\frac{1}{4!}F_5^2\right]
$$

Metric ansatz:
$$
ds_{12}^2=H^a(-dx_0^2 + dx_1^2 + ...+dx_5^2) + H^{-\frac{ad}{D-d-2}} (dy_0^2 + dy_1^2 + ... + dy_5^2)
$$

# 1. Compute Ricci tensor

parameters

In [1]:
D = 12
d = 6
HARMONIC_DIMENSIONS = D - d

set up coordinates

In [2]:
import sympy as sp
# coordinates
x = sp.symbols(f"x0:{d}", real=True)
y = sp.symbols(f"y0:{D - d}", real=True)
coords = (*x,*y)


# Parameters and symbols
apar = sp.symbols('a', real=True)
apar = -sp.Rational(3,5)
r_sym = sp.symbols('r', real=True, positive=True)

# expressions for radial coordinate r and r**2
r2 = sum(y_i**2 for y_i in y)
r = sp.sqrt(r2)

# harmonic function H, both as function of r, and as symbols
H = sp.Function('H')(r)
H_, H_p, H_pp = sp.symbols("H H' H''") # symbols for H and its derivatives, easier to work with once derivatives are taken and only manipulating algebraically

print(f"We are in {D} dimensions, with coordinates {coords}")
print(f"Radial coordinate r = {r}, r^2 = {r2}")
print(f"Harmonic function H is harmonic in {HARMONIC_DIMENSIONS} dimensions.")

We are in 12 dimensions, with coordinates (x0, x1, x2, x3, x4, x5, y0, y1, y2, y3, y4, y5)
Radial coordinate r = sqrt(y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2), r^2 = y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2
Harmonic function H is harmonic in 6 dimensions.


In [3]:
from helpers.einstein_helpers import minkowski_metric, euclidean_metric
# metric
g_MN = sp.zeros(D)
g_MN[0:d,0:d] = H**(apar) * minkowski_metric(d)
g_MN[d:D,d:D] = H**(-apar * sp.Rational(d, D -d -2)) * euclidean_metric(D - d)
g_MN = sp.simplify(g_MN)
g = sp.simplify(sp.sqrt(-(sp.det(g_MN))))
inv_g_MN = g_MN.inv()

In [4]:
from helpers.einstein_helpers import ricci_tensor
R_MN = ricci_tensor(g_MN, coords)

Computed Gamma^0_...,Computed Gamma^1_...,Computed Gamma^2_...,Computed Gamma^3_...,Computed Gamma^4_...,Computed Gamma^5_...,Computed Gamma^6_...,Computed Gamma^7_...,Computed Gamma^8_...,Computed Gamma^9_...,Computed Gamma^10_...,Computed Gamma^11_...,Computed R[0,...],Computed R[1,...],Computed R[2,...],Computed R[3,...],Computed R[4,...],Computed R[5,...],Computed R[6,...],Computed R[7,...],Computed R[8,...],Computed R[9,...],Computed R[10,...],Computed R[11,...],

# 2. Simplifying relevant terms

From this point on no more derivatives shall be taken

In [5]:
def harmonic_derivative_subs(expr):
    # First sub in r and r^2
    expr = expr.subs({r: r_sym,
                      r2: r_sym**2})
    expr = sp.simplify(expr)
    expr = sp.simplify(expr.subs({sp.sqrt(r2): r_sym,
                                  r2: r_sym**2}))
    
    # Then sub symbols of H, H', H'' for its literal derivatives
    expr = sp.simplify(expr.subs({sp.Function('H')(r_sym): H_,
                                  sp.Derivative(sp.Function('H')(r_sym), r_sym): H_p,
                                  sp.Derivative(sp.Function('H')(r_sym), (r_sym, 2)): H_pp}))
    
    # now impose harmonic condition to eliminate H''
    expr = expr.subs({H_pp: -(HARMONIC_DIMENSIONS -1)/r_sym * H_p})
    return sp.simplify(expr)

Obtain simplified expression for $R_{00}$

In [6]:
R_00 = harmonic_derivative_subs(R_MN[0,0])
R_00 = sp.factor(R_00).subs(r**2, r_sym**2)
R_00

3*H'**2/(10*H**(7/2))

In [7]:
R_66 = harmonic_derivative_subs(R_MN[6,6])
R_66 = R_66.subs(sum([yi**2 for yi in y[1:]]), r_sym**2 - y[0]**2)
R_66

9*H'**2*(r**2 - 3*y0**2)/(20*H**2*r**2)

# 3. form field

We are going to try

$$
(F_7)_{m 0 1 2 3 4 5} = c_1 H^{d_1} \partial_m H
= c_1 H^{d_1} \frac{y^m}{r}H'
$$

and something else for its magnetic components which will involve $c_2, d_2$

In [128]:
c_1, c_2= sp.symbols('c_1 c_2')
d_1, d_2= sp.symbols('d_1 d_2', real=True)

# These are required to balance the Einstein equations
d_1 = sp.Rational(-14,5)
d_2 = sp.Rational(4,5)

In [129]:
from helpers.form_helpers import antisymmetrize_tensor
F7 = sp.MutableDenseNDimArray.zeros(*([D] * 7))

for m in range(6):
    # F[m+4,0,1,2,3] = c_1 * H**d_1 * sp.Derivative(H**(-1), y[m])
    F7[m+6,0,1,2,3,4,5] = c_1 * H_**(d_1) * y[m] / r_sym * H_p

F7 = antisymmetrize_tensor(F7)

In [130]:
F7[4,5,6,7,8,9,10]

0

In [131]:
F5 = sp.MutableDenseNDimArray.zeros(*([D] * 5))

F5[6,7,8,9,10] = c_2 * H_**(d_2) * coords[11] / r_sym * H_p
F5[7,8,9,10,11] = - c_2 * H_**(d_2) * coords[6] / r_sym * H_p
F5[8,9,10,11,6] = c_2 * H_**(d_2) * coords[7] / r_sym * H_p
F5[9,10,11,6,7] = - c_2 * H_**(d_2) * coords[8] / r_sym * H_p
F5[10,11,6,7,8] = c_2 * H_**(d_2) * coords[9] / r_sym * H_p
F5[11,6,7,8,9] = -c_2 * H_**(d_2) * coords[10] / r_sym * H_p

F5 = antisymmetrize_tensor(F5)

In [132]:
F5[6,11,7,8,0]

0

In [81]:
from helpers.form_helpers import hodge_dual

F5_dual = hodge_dual(F5, g_MN)

In [82]:
harmonic_derivative_subs(F5_dual[4,5,6,7,8,9,10])

0

In [133]:
from helpers.form_helpers import compute_FF_MN, compute_S_MN, compute_form_squared
S5_MN = compute_S_MN(g_MN, 4, compute_FF_MN(F5, inv_g_MN), compute_form_squared(F5, inv_g_MN))
S7_MN = compute_S_MN(g_MN, 6, compute_FF_MN(F7, inv_g_MN), compute_form_squared(F7, inv_g_MN))

Computing F_M PQRS F_N^PQRS for a rank 5 form in 12 dimensions.
Computing squared norm of a rank 5 form in 12 dimensions.
Computing F_M PQRS F_N^PQRS for a rank 7 form in 12 dimensions.
Computing squared norm of a rank 7 form in 12 dimensions.


In [134]:
S_MN = S7_MN + S5_MN

In [141]:
S_00 = harmonic_derivative_subs(S_MN[0,0])
S_00

H'**2*(5*c_1**2 + 3*c_2**2)/(4*H**(7/2))

In [147]:
S_66 = harmonic_derivative_subs(S_MN[6,6])
S_66 = S_66.subs({
    sum([5 * yi**2 for yi in y[1:]]): 5 * (r_sym**2-y[0]**2),
    sum([7 * yi**2 for yi in y[1:]]): 7 * (r_sym**2-y[0]**2),
}).simplify()
S_66

H'**2*(c_1**2*(7*r**2 - 12*y0**2) + c_2**2*(5*r**2 - 8*y0**2))/(4*H**2*r**2)

# 4. trying to balance the equations

Trying to match the equations, we find $d_1 = -\frac{6}{5}, d_2 = -\frac{1}{5}$

In [151]:
R_66

9*H'**2*(r**2 - 3*y0**2)/(20*H**2*r**2)

In [152]:
S_66

H'**2*(c_1**2*(7*r**2 - 12*y0**2) + c_2**2*(5*r**2 - 8*y0**2))/(4*H**2*r**2)

In [148]:
from helpers.sympy_helpers import find_common_factor
common_factor_00 = find_common_factor(R_00, S_00)
eq00 = sp.Eq(R_00/common_factor_00, S_00/common_factor_00)
eq00

PolynomialError: H**(7/2) contains an element of the set of generators.

In [19]:
sp.solve(eq00, c_1**2)

[6/25 - 7*c_2**2/5]

In [20]:
common_factor_44 = find_common_factor(R_44, S_44)
eq44 = sp.Eq(R_44/common_factor_44, S_44/common_factor_44)
eq44

Common factor found: H'**2/(H**2*r**2)


Eq(r**2/5 - 3*y0**2/5, 3*c_1**2*r**2/4 - 2*c_1**2*y0**2 + 5*c_2**2*r**2/4 - 3*c_2**2*y0**2)

In [21]:
eq44_subbed = eq44.subs({c_1**2: 6/25 - 7*c_2**2/5})

In [22]:
eq44_subbed

Eq(r**2/5 - 3*y0**2/5, 5*c_2**2*r**2/4 - 3*c_2**2*y0**2 + 3*r**2*(0.24 - 7*c_2**2/5)/4 - 2*y0**2*(0.24 - 7*c_2**2/5))

In [23]:
sp.solve(eq44_subbed, c_2**2)

[0.1*(r**2 - 6.0*y0**2)/(r**2 - y0**2)]

In [24]:
find_common_factor(R_00, S_00)

Common factor found: H'**2/(2*H**3)


H'**2/(2*H**3)

In [25]:
e1 = sp.cancel(R_44)   # or eq00.lhs
e2 = sp.cancel(S_44)   # or eq00.rhs

n1, d1 = sp.fraction(sp.together(e1))
n2, d2 = sp.fraction(sp.together(e2))

# choose generators you care about (recommended, avoids huge gens lists)
gens = [H_, H_p, r_sym, c_1, c_2]   # adjust as needed

g_num = sp.gcd(sp.Poly(n1, *gens), sp.Poly(n2, *gens)).as_expr()
g_den = sp.gcd(sp.Poly(d1, *gens), sp.Poly(d2, *gens)).as_expr()

common = sp.simplify(g_num / g_den)
common

H'**2/(H**2*r**2)

In [26]:
sp.factor(R_44)

-H'**2*(-r**2 + 3*y0**2)/(5*H**2*r**2)

In [27]:
S_44.expand()

3*H'**2*c_1**2/(4*H**2) - 2*H'**2*c_1**2*y0**2/(H**2*r**2) + 5*H'**2*c_2**2/(4*H**2) - 3*H'**2*c_2**2*y0**2/(H**2*r**2)

In [28]:
sp.factor(eq00.lhs,

_IncompleteInputError: incomplete input (2856581245.py, line 1)

In [ ]:
sp.solve([eq00, eq44], (c_1, c_2))

[(-sqrt(10)*sqrt(-1/((-r + y0)*(r + y0)))*sqrt(r**2 + 6*y0**2)/12,
  -sqrt(10)*sqrt((-r**2 + 6*y0**2)/((-r + y0)*(r + y0)))/6),
 (-sqrt(10)*sqrt(-1/((-r + y0)*(r + y0)))*sqrt(r**2 + 6*y0**2)/12,
  sqrt(10)*sqrt((-r**2 + 6*y0**2)/((-r + y0)*(r + y0)))/6),
 (sqrt(10)*sqrt(-1/((-r + y0)*(r + y0)))*sqrt(r**2 + 6*y0**2)/12,
  -sqrt(10)*sqrt((-r**2 + 6*y0**2)/((-r + y0)*(r + y0)))/6),
 (sqrt(10)*sqrt(-1/((-r + y0)*(r + y0)))*sqrt(r**2 + 6*y0**2)/12,
  sqrt(10)*sqrt((-r**2 + 6*y0**2)/((-r + y0)*(r + y0)))/6)]

In [ ]:
sp.cancel(eq00)

Eq(3*H'**2/(10*H(r)**3), 9*H'**2*(20*c_1**2 + 7*c_2**2)/(100*H(r)**3))

In [ ]:
eq44

Eq(H'**2*(r**2 - 3*y0**2)/(5*r**2*H(r)**2), -9*H'**2*(20*c_1**2*y0**2 - 12*c_1**2*(r**2 - y0**2) + 7*c_2**2*y0**2 - 5*c_2**2*(r**2 - y0**2))/(100*r**2*H(r)**2))